In [1]:
using Gadfly
using JuMP
using Ipopt
using Interact

# Advanced Micro, Lecture 1
Nova School of Business and Economics  
February 15, 2018  

---

_Deriving the utility of agents A and B given initial endowments._

In [2]:
# Endowment of agent A
ω1_A = 6
ω2_A = 8

# Endowment of agent B
ω1_B = 4
ω2_B = 12;

In [3]:
# Utility functions of agents A and B

U_A(x1, x2) = x1*x2
U_B(x1, x2) = x1*x2

println("Utility of Agent A = ", U_A(ω1_A,ω2_A))
println("Utility of Agent B = ", U_B(ω1_B,ω2_B))

Utility of Agent A = 48
Utility of Agent B = 48


---


## Edgeworth Box

1. Tiago's way: allows for redrawing and changing the dimension of the Edgeworth Box
2. My way: focus on derivation of contract curve

---

### 1. Tiago's Way

In [6]:
# Making empty lists for the domain and range of the function
domain_IC_A = []
range_IC_A = [];

In [10]:
# Making a function using the Ipopt functionality
function IC_A_max(utility, endowment1=(ω1_A+ω1_B), endowment2=(ω2_A+ω2_B))    
    IC_A = Model(solver=IpoptSolver())
    
    @variable(IC_A, x1_A >= 0 )
    @variable(IC_A, x2_A >= 0)
    @variable(IC_A, x1_B >= 0 )
    @variable(IC_A, x2_B >= 0)
    
    @constraint(IC_A, x1_B * x2_B >= utility)
    @constraint(IC_A, x1_A + x1_B == endowment1)
    @constraint(IC_A, x2_A + x2_B == endowment2)
    
    @objective(IC_A, Max, U_A(x1_A,x2_A))
    
    solve(IC_A)
    
    # Append the list using the push! functionality
    push!(domain_IC_A, getvalue(x1_A))   
    push!(range_IC_A, getvalue(x2_A))
    
end

IC_A_max (generic function with 3 methods)

In [ ]:
# Loop - solving for different levels of utility
for utility_level in 1:10:200
    IC_A_max(utility_level,10,20)
end

In [ ]:
domain = linspace(0, ω1_A+ω1_B, 1000);

In [ ]:
plot_U_A(x,x1,x2) = U_A(x1, x2)/x
plot_U_B(x,x1,x2) = U_B(x1, x2)/(x-ω1_A-ω1_B)+ω2_A+ω2_B

In [ ]:
set_default_plot_size(20cm, 20cm)

In [ ]:
plot(
    layer(x=[ω1_A], y=[ω2_A], Geom.point, Theme(default_color=colorant"black")),
    # initial curves
    layer(x=domain, y=plot_U_A(domain,ω1_A,ω2_A), Geom.line, Theme(default_color=colorant"black")),
    layer(x=domain, y=plot_U_B(domain,ω1_B,ω2_B), Geom.line, Theme(default_color=colorant"black")),
    # upper curves
    layer(x=domain, y=plot_U_A(domain,domain_CC[1],range_CC[1]), Geom.line, Theme(default_color=colorant"orange")),
    layer(x=domain, y=plot_U_B(domain,ω1_A+ω1_B-domain_CC[1],ω2_A+ω2_B-range_CC[1]), Geom.line, Theme(default_color=colorant"orange")),
    # lower curves
    layer(x=domain, y=plot_U_A(domain,domain_CC[15],range_CC[15]), Geom.line, Theme(default_color=colorant"orange")),
    layer(x=domain, y=plot_U_B(domain,ω1_A+ω1_B-domain_CC[15],ω2_A+ω2_B-range_CC[15]), Geom.line, Theme(default_color=colorant"orange")), 
    # contract curve
    layer(x=domain_CC, y=range_CC, Geom.point, Geom.line, Theme(default_color=colorant"grey")),
    Coord.Cartesian(xmin=0,xmax=ω1_A+ω1_B,ymin=0,ymax=ω2_A+ω2_B,fixed=true),
    Guide.xlabel("Good 1"),
    Guide.ylabel("Good 2"),
)

---

### 2. My Way

$$U^A = {(x_1^A)}^\alpha {(x_2^A)}^\beta \quad \quad U^B = {(x_1^B)}^\gamma {(x_2^B)}^\theta $$

### Endowments

Initial endowments must be given. Regardless of choice of points, these endowments will always be the same, such that:

$$ x_1^A + x_1^B = \omega_1^A + \omega_1^B \iff E_1 = x_1^A + x_1^B $$
$$ x_2^A + x_2^B = \omega_2^A + \omega_2^B \iff E_2 = x_2^A + x_2^B $$

Moreover, note that the utility of Agent B may be rewritten as:

$$ U^B = {(E_1-x_1^A)}^\gamma {(E_2-x_2^A)}^\theta $$

Setting the initial endowments below:

In [ ]:
e1 = 10
e2 = 20;

In [ ]:
# Setting up the utility functions for Agents A and B
U_A(x1, x2) = (x1^α)*(x2^β)
U_B(x1, x2) = ((e1-x1)^γ)*((e2-x1)^θ)

We apply the following transformation below in order to plot the indifference curves of Agents A and B in terms of $E_1$, $E_2$, as well as the points $x_1^A$ and $x_2^A$:

$$ U^A = {(x_1^A)}^\alpha {(x_2^A)}^\beta \quad \iff \quad {(x_2^A)}^\beta = \dfrac{U^A}{{(x_1^A)}^\alpha} \quad \iff \quad x_2^A = {\dfrac{U^A}{{(x_1^A)}^\alpha}}^{\beta^{-1}} $$



$$ U^B = {(x_1^B)}^\gamma {(x_2^B)}^\theta \quad \iff \quad {(x_2^B)}^\gamma = \dfrac{U^B}{{(x_1^B)}^\theta} \quad \iff \quad x_2^A = E_2 - {\dfrac{U^B}{{(E_1-x_1^A)}^\theta}}^{\gamma^{-1}} $$

In [ ]:
plot_U_A(x, x1, x2) = ( U_A(x1, x2)/ x^α ) ^ (1/β)
plot_U_B(x, x1, x2) = - (U_B(x1, x2) / (x - ω1A - ω1B)) - ω2A - ω2B